# MLP for Airline Passenger Satisfaction Dataset 

## Importing the Data

In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and merge datasets
df_train = pd.read_csv('all_datasets/Airline_passenger_satiscation_train.csv')
df_test = pd.read_csv('all_datasets/Airline_passenger_satiscation_test.csv')
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.drop(columns=["id"])

In [16]:
# Label encode categorical columns
categorical_cols = ["Gender", "Customer Type", "Type of Travel", "Class"]
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [17]:
target_encoder = LabelEncoder()
df["satisfaction"] = target_encoder.fit_transform(df["satisfaction"])

In [18]:
# Fill missing values
df = df.fillna(0)

In [19]:
# Feature/target split
X = df.drop(columns=["satisfaction"]).values
y = df["satisfaction"].values

In [20]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [21]:
# Define model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, output_size),
        )

    def forward(self, x):
        return self.model(x)


split_ratios = [0.5, 0.3, 0.2, 0.1]

In [24]:
for split in split_ratios:
    print(f"\n--- Evaluating {int((1 - split) * 100)}/{int(split * 100)} Train/Test Split ---")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=42, stratify=y)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    input_dim = X.shape[1]
    output_dim = len(np.unique(y))
    model = MLP(input_dim, output_dim)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    # Train model
    epochs = 300
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = loss_fn(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Evaluate model
    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_test_tensor), dim=1).numpy()
        y_true = y_test_tensor.numpy()

        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds, zero_division=0)
        f1 = f1_score(y_true, preds, zero_division=0)

        print(f"Accuracy:  {acc:.4f}")
        print(f"Precision: {prec:.4f}")
        print(f"Recall:    {rec:.4f}")
        print(f"F1 Score:  {f1:.4f}")



--- Evaluating 50/50 Train/Test Split ---
Accuracy:  0.9590
Precision: 0.9710
Recall:    0.9336
F1 Score:  0.9519

--- Evaluating 70/30 Train/Test Split ---
Accuracy:  0.9606
Precision: 0.9693
Recall:    0.9391
F1 Score:  0.9539

--- Evaluating 80/20 Train/Test Split ---
Accuracy:  0.9608
Precision: 0.9684
Recall:    0.9404
F1 Score:  0.9542

--- Evaluating 90/10 Train/Test Split ---
Accuracy:  0.9601
Precision: 0.9698
Recall:    0.9374
F1 Score:  0.9533


## Testing out different MLPs

In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and merge datasets
df_train = pd.read_csv('all_datasets/Airline_passenger_satiscation_train.csv')
df_test = pd.read_csv('all_datasets/Airline_passenger_satiscation_test.csv')
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.drop(columns=["id"])

# Encode categorical columns
categorical_cols = ["Gender", "Customer Type", "Type of Travel", "Class"]
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

# Encode target
target_encoder = LabelEncoder()
df["satisfaction"] = target_encoder.fit_transform(df["satisfaction"])

# Fill missing values and prepare data
df = df.fillna(0)
X = df.drop(columns=["satisfaction"]).values
y = df["satisfaction"].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Fixed 70/30 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Define MLP architectures
input_dim = X.shape[1]
output_dim = len(np.unique(y))

model_architectures = [
    nn.Sequential(
        nn.Linear(input_dim, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, output_dim),
    ),
    nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, output_dim),
    ),
    nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, output_dim),
    ),
    nn.Sequential(
        nn.Linear(input_dim, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, output_dim),
    )
]

# Store results
results = []

# Train and evaluate each model
for i, architecture in enumerate(model_architectures, 1):
    class MLP(nn.Module):
        def __init__(self):
            super().__init__()
            self.model = architecture

        def forward(self, x):
            return self.model(x)


    model = MLP()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.002)

    # Train
    for epoch in range(300):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = loss_fn(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Evaluate
    model.eval()
    with torch.no_grad():
        preds = torch.argmax(model(X_test_tensor), dim=1).numpy()
        y_true = y_test_tensor.numpy()

        acc = accuracy_score(y_true, preds)
        prec = precision_score(y_true, preds, zero_division=0)
        rec = recall_score(y_true, preds, zero_division=0)
        f1 = f1_score(y_true, preds, zero_division=0)

        results.append((f"MLP{i}", acc, prec, rec, f1))

# Print results
print("\nMODEL\tAccuracy\tPrecision\tRecall\t\tF1-Score")
for model_name, acc, prec, rec, f1 in results:
    print(f"{model_name}\t{acc:.4f}\t\t{prec:.4f}\t\t{rec:.4f}\t\t{f1:.4f}")


MODEL	Accuracy	Precision	Recall		F1-Score
MLP1	0.9511		0.9554		0.9309		0.9430
MLP2	0.9575		0.9675		0.9336		0.9502
MLP3	0.9571		0.9570		0.9437		0.9503
MLP4	0.9630		0.9724		0.9415		0.9567


## Cross Validation
using the best model from the previous section

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load and preprocess dataset
df_train = pd.read_csv('all_datasets/Airline_passenger_satiscation_train.csv')
df_test = pd.read_csv('all_datasets/Airline_passenger_satiscation_test.csv')
df = pd.concat([df_train, df_test], ignore_index=True)
df = df.drop(columns=["id"])

# Encode categorical features
categorical_cols = ["Gender", "Customer Type", "Type of Travel", "Class"]
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

# Encode target
target_encoder = LabelEncoder()
df["satisfaction"] = target_encoder.fit_transform(df["satisfaction"])

# Prepare data
df = df.fillna(0)
X = df.drop(columns=["satisfaction"]).values
y = df["satisfaction"].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)


# Custom MLP with specified architecture
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

        def forward(self, x):
            return self.model(x)

    # Cross-validation function (CPU only)
    def run_cv(X, y, n_splits):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        input_dim = X.shape[1]
        output_dim = len(np.unique(y))
        results = []

        for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
            print(f"\n[Fold {fold}/{n_splits}]")

            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
            y_train_tensor = torch.tensor(y_train, dtype=torch.long)
            X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
            y_test_tensor = torch.tensor(y_test, dtype=torch.long)

            model = MLP(input_dim, output_dim)
            loss_fn = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.002)

            for epoch in range(300):
                model.train()
                optimizer.zero_grad()
                outputs = model(X_train_tensor)
                loss = loss_fn(outputs, y_train_tensor)
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                preds = model(X_test_tensor).argmax(dim=1).numpy()
                y_true = y_test_tensor.numpy()

                acc = accuracy_score(y_true, preds)
                prec = precision_score(y_true, preds, zero_division=0)
                rec = recall_score(y_true, preds, zero_division=0)
                f1 = f1_score(y_true, preds, zero_division=0)

                results.append((acc, prec, rec, f1))

        return results

    # Run 5-fold and 10-fold CV
    results_5 = run_cv(X, y, 5)
    results_10 = run_cv(X, y, 10)

    # Print results
    def print_results(results, label):
        accs, precs, recs, f1s = zip(*results)
        print(f"\n=== {label} Results ===")
        print("MODEL\tAccuracy\tPrecision\tRecall\t\tF1-Score")
        print(f"{label}\t{np.mean(accs):.4f}\t\t{np.mean(precs):.4f}\t\t{np.mean(recs):.4f}\t\t{np.mean(f1s):.4f}")

    print_results(results_5, "5-Fold")
    print_results(results_10, "10-Fold")


[Fold 1/5]

[Fold 2/5]

[Fold 3/5]

[Fold 4/5]

[Fold 5/5]

[Fold 1/10]

[Fold 2/10]

[Fold 3/10]


KeyboardInterrupt: 

=== 5-Fold Results ===
MODEL    Accuracy    Precision    Recall        F1-Score
5-Fold    0.9648        0.9750        0.9433        0.9589

=== 10-Fold Results ===
MODEL    Accuracy    Precision    Recall        F1-Score
10-Fold    